<a href="https://colab.research.google.com/github/ngyxntthaoo/IS6404.CH201-Phan-tich-du-lieu-nang-cao/blob/main/Lab_4_Traditional_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4: Tổng quan các kỹ thuật phân tích dữ liệu cổ điển
**Course:** IS6404 - Advanced Data Analysis, UIT, VNUHCM  
**Lecturer:** Dr. Tran Hung Nghiep, 2026  

---

Mục tiêu của buổi này không phải học sâu công thức, mà là:
- Biết **khi nào dùng** các kỹ thuật cổ điển (regression, hypothesis testing, optimization)
- Biết dùng chúng như **baseline** và **công cụ kiểm chứng** trong pipeline ML/AI
- Hiểu các **hạn chế** (assumptions, leakage, p-hacking, overfitting, constraint mismatch)

> Notebook này có 3 phần chính: **Hồi quy**, **Kiểm chứng giả thuyết**, **Tối ưu**.  
> Mỗi phần đều có câu hỏi để người học trả lời.

## 0. Setup & Reproducibility
Cố định seed, in versions, tạo thư mục outputs.

In [ ]:
# 0) Setup
import json, os, sys, platform
from datetime import datetime

import numpy as np
import pandas as pd
import sklearn

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("python:", sys.version)
print("platform:", platform.platform())
print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("sklearn:", sklearn.__version__)


## 1. Dataset (dữ liệu mẫu cho regression + hypothesis testing)

Ta dùng dataset `diabetes` từ scikit-learn (có sẵn, chạy nhanh).
- Target: tiến triển bệnh (một đại lượng liên tục) -> phù hợp regression
- Có feature `sex` (đã chuẩn hóa) -> tạo điều kiện làm kiểm chứng giả thuyết giữa 2 nhóm

> Trong đồ án thật, hãy giữ cấu trúc notebook và thay bằng dataset của nhóm.


In [ ]:
from sklearn.datasets import load_diabetes

data = load_diabetes(as_frame=True)
df = data.frame.copy()
df.rename(columns={"target": "y"}, inplace=True)

df.head()


In [ ]:
print("shape:", df.shape)
df.describe().T.head(12)


## 2. Hồi quy tuyến tính (Linear Regression) như baseline + interpretability

### Vai trò trong pipeline ML/AI
- Baseline mạnh, dễ giải thích
- Kiểm tra nhanh mối quan hệ tuyến tính và multicollinearity
- Tạo feature/insight trước khi dùng model phức tạp

### Cảnh báo
- Assumptions (tuyến tính, độc lập, phương sai đồng nhất, sai số gần chuẩn) thường **không hoàn hảo**
- Không suy ra nhân quả chỉ từ hồi quy

### **Câu hỏi**
1) Với dữ liệu của nhóm bạn, hồi quy tuyến tính có thể là baseline không? Vì sao?
2) Nếu feature có quan hệ phi tuyến, dấu hiệu nào cho thấy OLS không phù hợp?


### 2.1 Train/Test split (để tránh đánh giá ảo)

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["y"])
y = df["y"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED
)
X_train.shape, X_test.shape


### 2.2 Linear regression bằng scikit-learn

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

linreg = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

linreg.fit(X_train, y_train)
pred = linreg.predict(X_test)

mae = mean_absolute_error(y_test, pred)
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, pred)

{"test_MAE": mae, "test_RMSE": rmse, "test_R2": r2}


### 2.3 Hồi quy OLS bằng statsmodels để xem hệ số và p-values

In [ ]:
import statsmodels.api as sm

X_train_sm = sm.add_constant(X_train)  # thêm intercept
ols = sm.OLS(y_train, X_train_sm).fit()
ols.summary()


**Gợi ý đọc kết quả OLS (đơn giản):**
- `coef`: hướng và độ mạnh ảnh hưởng (theo mô hình tuyến tính)
- `P>|t|`: p-value của giả thuyết hệ số = 0 (cẩn thận diễn giải)
- `R-squared`: mức giải thích biến thiên (không phải thước đo duy nhất)

> Trong ML hiện đại, p-value không phải lúc nào cũng là trọng tâm, nhưng nó hữu ích để **kiểm tra nhanh** và **diễn giải**.


### 2.4 Chẩn đoán nhanh: residuals (dấu hiệu phi tuyến / outliers)

In [ ]:
import matplotlib.pyplot as plt

pred_train = ols.predict(sm.add_constant(X_train))
resid = y_train - pred_train

plt.figure()
plt.scatter(pred_train, resid, s=15)
plt.axhline(0)
plt.xlabel("Predicted (train)")
plt.ylabel("Residual")
plt.title("Residual plot (train)")
plt.show()


## 3. Hồi quy phi tuyến (Non-linear regression) dưới dạng feature engineering

Thay vì đi sâu các mô hình phi tuyến cổ điển, ta dùng cách thực dụng:
- **Polynomial features + regularization** để bắt quan hệ phi tuyến
- So sánh với OLS baseline

### Vai trò trong pipeline
- Tạo baseline phi tuyến đơn giản trước khi dùng tree/NN
- Cho thấy lợi ích của feature engineering

### **Câu hỏi**
1) Polynomial degree tăng sẽ gây rủi ro gì?
2) Vì sao regularization (Ridge/Lasso) thường cần khi thêm polynomial features?


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

poly_ridge = Pipeline([
    ("scaler", StandardScaler()),
    ("poly", PolynomialFeatures(include_bias=False)),
    ("model", Ridge(random_state=RANDOM_SEED))
])

param_grid = {
    "poly__degree": [1, 2, 3],
    "model__alpha": [0.1, 1.0, 10.0, 100.0],
}

gs = GridSearchCV(poly_ridge, param_grid, cv=5, scoring="neg_mean_absolute_error")
gs.fit(X_train, y_train)

best = gs.best_estimator_
pred = best.predict(X_test)

mae = mean_absolute_error(y_test, pred)
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, pred)

gs.best_params_, {"test_MAE": mae, "test_RMSE": rmse, "test_R2": r2}


## Minh hoạ assumption violation gây sai lệch

- heteroscedasticity
- multicollinearity
- overfitting khi degree tăng

In [ ]:
from sklearn.metrics import mean_absolute_error

for deg in [1,2,3,5,8]:
    model = Pipeline([
        ("scaler", StandardScaler()),
        ("poly", PolynomialFeatures(degree=deg, include_bias=False)),
        ("lr", LinearRegression())
    ])
    model.fit(X_train, y_train)
    print("degree", deg, "MAE", mean_absolute_error(y_test, model.predict(X_test)))


## 4. Kiểm chứng giả thuyết (Hypothesis Testing) để kiểm tra claim

### Vai trò trong pipeline ML/AI
- Kiểm tra khác biệt giữa nhóm (A/B, cohort, segment)
- Kiểm tra feature có liên hệ đáng kể với target không
- Hỗ trợ data understanding và error analysis

### Giới hạn và rủi ro
- p-value không đo “mức độ quan trọng thực tế” (cần effect size, CI)
- Nhiều phép thử -> tăng false positives (p-hacking)
- Giả định phân phối có thể không đúng -> cân nhắc permutation/bootstrapping

### **Câu hỏi**
1) p-value nhỏ nghĩa là gì? p-value không có nghĩa là gì?
2) Vì sao cần effect size/CI bên cạnh p-value?


### 4.1 Tạo 2 nhóm theo feature `sex`
Trong dataset này, `sex` đã chuẩn hóa quanh 0. Ta tách nhóm theo dấu.

In [ ]:
df2 = df.copy()
df2["sex_group"] = np.where(df2["sex"] >= 0, "group_A", "group_B")

df2["sex_group"].value_counts(), df2.groupby("sex_group")["y"].mean()


### 4.2 Two-sample t-test (so sánh trung bình y giữa 2 nhóm)

In [ ]:
from scipy import stats

a = df2.loc[df2["sex_group"] == "group_A", "y"].values
b = df2.loc[df2["sex_group"] == "group_B", "y"].values

t_stat, p_val = stats.ttest_ind(a, b, equal_var=False)  # Welch's t-test
t_stat, p_val


### 4.3 Effect size (Cohen's d) + confidence interval (bootstrap)

In [ ]:
def cohens_d(x, y):
    nx, ny = len(x), len(y)
    vx, vy = np.var(x, ddof=1), np.var(y, ddof=1)
    pooled = ((nx-1)*vx + (ny-1)*vy) / (nx+ny-2)
    return (np.mean(x) - np.mean(y)) / np.sqrt(pooled)

d = cohens_d(a, b)
d


In [ ]:
# Bootstrap CI for mean difference
def bootstrap_mean_diff(x, y, n=5000, seed=RANDOM_SEED):
    rng = np.random.default_rng(seed)
    diffs = []
    for _ in range(n):
        xb = rng.choice(x, size=len(x), replace=True)
        yb = rng.choice(y, size=len(y), replace=True)
        diffs.append(np.mean(xb) - np.mean(yb))
    return np.percentile(diffs, [2.5, 50, 97.5])

ci = bootstrap_mean_diff(a, b)
ci  # [2.5%, 50%, 97.5%]


### 4.4 Permutation test (khi muốn giảm phụ thuộc vào giả định phân phối)

In [ ]:
def permutation_test_mean_diff(x, y, n=5000, seed=RANDOM_SEED):
    rng = np.random.default_rng(seed)
    obs = np.mean(x) - np.mean(y)
    pooled = np.concatenate([x, y]).copy()
    nx = len(x)
    diffs = []
    for _ in range(n):
        rng.shuffle(pooled)
        diffs.append(np.mean(pooled[:nx]) - np.mean(pooled[nx:]))
    diffs = np.array(diffs)
    p = (np.abs(diffs) >= abs(obs)).mean()
    return obs, p

obs, p_perm = permutation_test_mean_diff(a, b)
obs, p_perm


## Minh hoạ về multiple testing và p-hacking

In [ ]:
from scipy import stats

pvals = []
for col in X.columns:
    r, p = stats.pearsonr(X[col], y)
    pvals.append((col, p))

sorted(pvals, key=lambda x: x[1])[:50]


## 5. Tối ưu (Optimization) như một module trong hệ thống dữ liệu

### Vai trò trong pipeline ML/AI
- Sau khi dự báo/ước lượng (ML), ta cần **ra quyết định** (optimization)
- Ví dụ:
  - dự báo nhu cầu -> tối ưu tồn kho
  - dự báo churn -> tối ưu phân bổ ngân sách giữ chân
  - scoring khách hàng -> tối ưu chọn top-K theo ràng buộc

### Bài lab: 2 bài toán minh họa
A) Linear Programming: tối ưu product mix (tuyến tính, ràng buộc tuyến tính)  
B) Non-linear Optimization: tối ưu giá (có độ cong, không tuyến tính)

### **Câu hỏi**
1) Vì sao optimization thường đứng sau ML trong hệ thống thực?
2) Khi nào mô hình tối ưu tuyến tính không phù hợp?


### 5A. Linear Programming (LP): Product mix optimization
Bài toán: chọn sản lượng 3 sản phẩm để tối đa lợi nhuận, bị giới hạn bởi tài nguyên.

In [ ]:
import numpy as np
from scipy.optimize import linprog

# Decision variables: x1, x2, x3 (sản lượng 3 sản phẩm)
profit = np.array([30, 20, 25])  # lợi nhuận / đơn vị

# Constraints (Ax <= b)
# Resource 1: 2x1 + 1x2 + 1x3 <= 100
# Resource 2: 1x1 + 3x2 + 2x3 <= 120
A = np.array([
    [2, 1, 1],
    [1, 3, 2]
])
b = np.array([100, 120])

bounds = [(0, None), (0, None), (0, None)]

res = linprog(c=-profit, A_ub=A, b_ub=b, bounds=bounds, method="highs")
res.success, res.message


In [ ]:
x = res.x
max_profit = profit @ x
{"x1": x[0], "x2": x[1], "x3": x[2], "max_profit": max_profit}


### 5B. Non-linear optimization: Pricing
Giả sử nhu cầu giảm theo giá theo hàm mũ. Tối ưu giá để tối đa lợi nhuận.

In [ ]:
from scipy.optimize import minimize

# Demand model: q(p) = a * exp(-b*p)
a, b = 1000, 0.05
unit_cost = 5.0  # chi phí đơn vị

def demand(p):
    return a * np.exp(-b * p)

def neg_profit(p_vec):
    p = p_vec.item()
    q = demand(p)
    profit = (p - unit_cost) * q
    return -profit

res2 = minimize(neg_profit, x0=np.array([30.0]), bounds=[(0.0, 200.0)])
res2.success, res2.x, -res2.fun


In [ ]:
p_star = res2.x.item()
q_star = float(demand(p_star))
profit_star = (p_star - unit_cost) * q_star
{"p_star": p_star, "q_star": q_star, "profit_star": profit_star}


### 5B.1 Visualize profit curve (để hiểu tối ưu)

In [ ]:
import matplotlib.pyplot as plt

ps = np.linspace(0, 200, 201)
profits = [(p - unit_cost) * demand(p) for p in ps]

plt.figure()
plt.plot(ps, profits)
plt.axvline(p_star)
plt.xlabel("price p")
plt.ylabel("profit")
plt.title("Profit vs Price")
plt.show()


## 6. Tổng kết: kỹ thuật cổ điển nằm ở đâu trong ML/AI pipeline?

- Regression: baseline + interpretability + sanity check
- Hypothesis testing: kiểm chứng claim + hiểu dữ liệu + A/B style reasoning
- Optimization: ra quyết định dưới ràng buộc, thường đứng sau ML


# Bài tập về nhà
1) Trả lời tất cả câu hỏi bắt buộc trong notebook (ngắn gọn, 3–6 dòng/câu).
2) Thay bài toán LP bằng một kịch bản liên quan đến project của nhóm bạn (tối thiểu 2 ràng buộc).
3) Với dữ liệu đồ án:
   - đề xuất 1 baseline hồi quy hoặc kiểm định giả thuyết phù hợp
   - mô tả rủi ro p-hacking hoặc leakage có thể gặp
